In [1]:
%%writefile app.py
# app.py
# Importamos librerías
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
import scipy.special as special
from funpymodeling.exploratory import freq_tbl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score


# Lista de usuarios (nombre de usuario y contraseña)
usuarios = [
    {"username": "ShadowPingPong", "password": "Borrego01"},
    {"username": "ClutchMasterX", "password": "Borrego02"},
    {"username": "ToxicHugz", "password": "Borrego03"},
    {"username": "SilentGee", "password": "Borrego04"}]

# Función para iniciar sesión
def iniciar_sesion():
    st.title("Inicio de Sesión")
    username = st.text_input("Nombre de usuario")
    password = st.text_input("Contraseña", type='password')

    # Botón para confirmar el inicio de sesión
    if st.button("Iniciar sesión"):
        # Para verificar que las credenciales coincidan con los usuarios registrados
        for usuario in usuarios:
            if usuario["username"] == username and usuario["password"] == password:
                st.session_state.authenticated = True # Para marcar al usuario como autenticado
                st.session_state.username = username
                st.success(f"Inicio de sesión exitoso. Bienvenido, {username}!")
                return # Finaliza la sesión si las credenciales son correctas 
        st.error("Nombre de usuario o contraseña incorrectos.")

# Inicializa el estado de la sesión si no existe 
if 'authenticated' not in st.session_state:
    st.session_state.authenticated = False # Establece el estado de autenticación como falso 

# Mostrar la página de inicio de sesión si no está autenticado
if not st.session_state.authenticated:
    iniciar_sesion()
    st.stop() # Detiene la ejecución del código hasta que se inicie sesión 

# Botón para cerrar sesión
if st.sidebar.button("Cerrar sesión"):
    st.session_state.authenticated = False
    st.session_state.username = None
    st.experimental_rerun()  # Refresca la aplicación para volver a la página de inicio de sesión

# Página principal del dashboard
st.title(f"Bienvenido al Dashboard, {st.session_state.username}")

# Definimos la instancia de streamlit
@st.cache_resource
# Creamos la función de carga de datos
def load_data_Seattle():
    # Carga desde un archivo csv con índice
    data = pd.read_csv('Seattle.csv', encoding='utf-8', index_col=0)



    # Etapa de procesamiento de Datos
    data_cuant = data.copy()
    # Se eliminan columnas innecesarias
    data_cuant.drop(["latitude", "longitude", "availability_30", "host_name",
                     "host_verifications", "neighbourhood_cleansed", "property_type", "room_type", "bathrooms_text",
                     "amenities","hostTotalListingsCat"], axis=1, inplace=True)

    # Seleccionar solo columnas numéricas
    data_cuant = data_cuant.select_dtypes(include=[np.number])

    # Encontramos todos los coeficientes de correlación entre las variables
    corr_factors = abs(data_cuant.corr())



    # Variables a analizar
    data_results = data_cuant[['host_response_rate', 'host_acceptance_rate',
                                'host_total_listings_count', 'accommodates',
                                'reviews_per_month', 'price']]


    # Selección de columnas numéricas del dataframe
    numeric_data = data_cuant.select_dtypes(['float', 'int'])
    numeric_cols = numeric_data.columns

    # Agregamos el DataFrame original para análisis de variables categóricas
    data_cat = data.copy()

    return data, data_cuant, corr_factors, data_results, numeric_data, numeric_cols, data_cat

# Cargo los datos obtenidos de la función 'load_data'
data, data_cuant, corr_factors, data_results, numeric_data, numeric_cols, data_cat = load_data_Seattle()

# Definición de las funciones de regresión
def funcCuad(x, a, b, c):
    return a * x**2 + b * x + c

def funcInversa(x, a):
    return 1 / a * x

def funcCuadraticaInversa(x, a):
    return 1 / a * x**2

# Página principal
st.title('S E A T T L E')
st.sidebar.subheader('Base de datos Seattle')
st.sidebar.subheader('Tipos de Análisis')


# Selección de análisis
analysis_type = st.sidebar.radio("Seleccione el tipo de análisis",
                                   ("Análisis de Correlación", "Regresión Simple", "Regresión Multiple", "Regresión No Lineal",
                                   "Regresión Logística", "Análisis Univariado de Variables Categóricas"))


# Imagen de Seattle
image_Seattle_url = "https://upload.wikimedia.org/wikipedia/commons/7/76/Vista_de_Seattle%2C_Washington%2C_Estados_Unidos%2C_2017-09-02%2C_DD_07-08_HDR.jpg"
st.image(image_Seattle_url, caption="Imagen de Seattle", use_column_width=True)

# Análisis de Correlación
if analysis_type == "Análisis de Correlación":
    st.header('Análisis de Correlación')

    # Checkbox para mostrar el heatmap
    heatmap_checkbox = st.checkbox(label="Mostrar Heatmap")

    if heatmap_checkbox:
        # Heat Map para todas las variables
        heatmap = px.imshow(corr_factors, text_auto=True, color_continuous_scale="BrBg",
                            title="Mapa de calor", width=800, height=600)
        st.plotly_chart(heatmap)

    # Checkbox para mostrar la matriz de correlación
    matrix_checkbox = st.checkbox(label="Mostrar matriz de correlación")  # Checkbox

    if matrix_checkbox:
        # Mostramos la matriz de correlación
        st.write(corr_factors)




# Regresión Múltiple
elif analysis_type == "Regresión Multiple":
  
  # Selección de variables independientes y dependiente desde la barra lateral
  st.sidebar.subheader('Variables Independientes (X)')
  y_var = st.sidebar.multiselect('Selecciona las variables independientes a incluir:', options=numeric_cols)  # Multiselect
  st.sidebar.subheader('Variable Dependiente')
  x_var = st.sidebar.selectbox(label='Seleccione Variable Y', options=numeric_cols) # Selectbox
  if y_var and x_var:
        # Preparación de los datos
        X = numeric_data[y_var]  # Usar el DataFrame filtrado
        y = numeric_data[x_var]   # Usar el DataFrame filtrado

        # Separar en conjunto de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

        # Crear el modelo de regresión lineal múltiple
        modelo = LinearRegression()

        # Entrenar el modelo
        modelo.fit(X_train, y_train)

        # Realizar predicciones en el conjunto de prueba
        y_pred = modelo.predict(X_test)

        # Calcular métricas
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        correlation_coef = np.corrcoef(y_test, y_pred)[0, 1]  # Coeficiente de correlación



        st.write('### Resultados del Análisis de Regresión Múltiple:')
        st.write('Coeficiente de Determinación (R²):', r2)
        st.write('Raíz del Error Cuadrático Medio (RMSE):', rmse)
        st.write('Coeficiente de Correlación:', correlation_coef)



        # Gráfico de dispersión
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, color='blue', label='Predicciones', alpha=0.5)
        plt.scatter(y_test, y_test, color='red', label='Valores Reales', alpha=0.5)  # Añadir valores reales
        plt.xlabel('Valores Reales')
        plt.ylabel('Valores Predichos')
        plt.title('Gráfico de Dispersión de Valores Reales vs Predicciones')
        plt.legend()
        st.pyplot(plt)

        # Calcular la matriz de correlación
        correlation_matrix = numeric_data.corr()
        st.write('### Matriz de Correlación:')
        st.write(correlation_matrix)




# Regresión Simple
elif analysis_type == "Regresión Simple":

   st.sidebar.subheader('Variables Independiente')
   x_var = st.sidebar.selectbox(label='Seleccione Variable X', options=numeric_cols)
   st.sidebar.subheader('Variable Dependiente')
   y_var = st.sidebar.selectbox(label='Seleccione Variable Y', options=numeric_cols) # Selectbox
   if y_var and x_var:
    X = numeric_data[[x_var]]  # Seleccionamos la variable independiente
    y = numeric_data[y_var]  # Seleccionamos la variable dependiente

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Crear el modelo de regresión lineal simple
    modelo = LinearRegression()

    # Entrenar el modelo
    modelo.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = modelo.predict(X_test)

    # Calcular métricas
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    correlation_coef = np.corrcoef(y_test, y_pred)[0, 1]  # Coeficiente de correlación

    # Mostrar resultados en la aplicación de Streamlit
    st.write('### Resultados del Análisis de Regresión Simple:')
    st.write('Coeficiente de Determinación (R²):', r2)
    st.write('Coeficiente de Correlación:', correlation_coef)

    # Gráfico de Línea: Valores Reales vs Predicciones
    st.subheader('Lineplot: Valores Reales vs Predicciones (Regresión Simple)')
    fig, ax = plt.subplots()
    ax.plot(y_test.values, label='Valores Reales', linestyle='--', marker='o')
    ax.plot(y_pred, label='Predicciones', linestyle='-', marker='x')
    ax.set_xlabel('Índice')
    ax.set_ylabel('Valores')
    ax.set_title('Comparación de Valores Reales vs Predicciones')
    ax.legend()
    st.pyplot(fig)

    # Gráfico de dispersión: Valores reales vs dispersiones
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, color='blue', label='Predicciones', alpha=0.5)
    plt.scatter(y_test, y_test, color='red', label='Valores Reales', alpha=0.5)  # Añadir valores reales
    plt.xlabel('Valores Reales')
    plt.ylabel('Valores Predichos')
    plt.title('Gráfico de Dispersión de Valores Reales vs Predicciones')
    plt.legend()
    st.pyplot(plt)





# Regresión No Lineal
elif analysis_type == "Regresión No Lineal":
    st.header('Análisis de Regresión No Lineal')

    # Selección de variables para graficar
    vars_result_selected = st.sidebar.selectbox(label='Variable Objetivo', options=data_results.columns)

    st.sidebar.subheader('Variable Independiente')
    x_var = st.sidebar.selectbox(label='Seleccione Variable X', options=numeric_cols)

    st.sidebar.subheader('Función para Graficar')
    func = st.sidebar.selectbox(label='Seleccione Funcion para graficar', options=['Cuadrática', 'Inversa','Cuadrática Inversa'])

    if func == 'Cuadrática':
        data_copy = data_cuant.copy()

        # Declaramos las variables dependientes e independientes para la regresión No Lineal
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]

        st.text('Función Cuadrática - y = ax^2 + bx + c')


        try:
            # Ajustamos los parámetros de la función curve_fit
            parametros, covs = curve_fit(funcCuad, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)

            # Predicciones
            y_pred = funcCuad(x, *parametros)
            data_copy['y_pred'] = y_pred

            # Calculamos el coeficiente de determinación del modelo
            r2 = abs(r2_score(y, y_pred))

            # Calculamos el coeficiente de correlación del modelo
            r = np.sqrt(abs(r2))

            # Mostramos el Coeficiente
            st.text('Coeficiente de determinacion (R^2):')
            st.text(str(r2))
            st.text('Coeficiente de Correlación:')
            st.text(str(r))

            # SCATTERPLOT
            # Despliegue de un scatter plot, definiendo las variables "X" y "Y"
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Cuadrática")
            st.plotly_chart(figureRNL)
        except Exception as e:
            st.text('No se puede graficar la grafica ' + str(e))

    if func == 'Inversa':
        data_copy = data_cuant.copy()
        # Declaramos las variables dependientes e independientes para la regresión No Lineal
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]

        st.text('Inversa - y = 1 / (a * x)') 
        try:
            parametros, covs = curve_fit(funcInversa, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)

            # Predicciones
            y_pred = funcInversa(x, *parametros)
            data_copy['y_pred'] = y_pred

            # Calculamos el coeficiente de determinación del modelo
            r2 = abs(r2_score(y, y_pred))

            # Calculamos el coeficiente de correlación del modelo
            r = np.sqrt(abs(r2))

            # Mostramos el Coeficiente
            st.text('Coeficiente de determinacion (R^2):')
            st.text(str(r2))
            st.text('Coeficiente de Correlación:')
            st.text(str(r))

            # SCATTERPLOT
            # Despliegue de un scatter plot, definiendo las variables "X" y "Y"
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Inversa")
            st.plotly_chart(figureRNL)
        except Exception as e:
            st.text('No se puede graficar la gráfica ' + str(e))

    if func == 'Cuadrática Inversa':
          data_copy = data_cuant.copy()
          # Declaramos las variables dependientes e independientes para la regresión No Lineal
          x = data_cuant[[str(x_var)]]
          y = data_cuant[[str(vars_result_selected)]]
          st.text('Función Cuadrática Inversa - y = 1 / a * x**2')
          try:
              parametros, covs = curve_fit(funcCuadraticaInversa, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)
              # Predicciones
              y_pred = funcCuadraticaInversa(x, *parametros)
              data_copy['y_pred'] = y_pred
              # Calculamos el coeficiente de determinación del modelo
              r2 = abs(r2_score(y, y_pred))
              # Calculamos el coeficiente de correlación del modelo
              r = np.sqrt(abs(r2))
              # Mostramos el Coeficiente
              st.text('Coeficiente de determinacion (R^2):')
              st.text(str(r2))
              st.text('Coeficiente de Correlación:')
              st.text(str(r))
              # SCATTERPLOT
              # Despliegue de un scatter plot, definiendo las variables "X" y "Y"
              figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Cuadrática Inversa")
              st.plotly_chart(figureRNL)
          except Exception as e:
              st.text('No se puede graficar la gráfica ' + str(e))





# Análisis Univariado de Variables Categóricas
elif analysis_type == "Análisis Univariado de Variables Categóricas":
    st.header('Análisis Univariado de Variables Categóricas')
       # Seleccionar las columnas categóricas del DataFrame
    # Seleccionar las columnas categóricas del DataFrame

    # Seleccionar las columnas categóricas del DataFrame
    categorical_cols = data_cat.select_dtypes(include=['object']).columns

    # Selectbox para elegir la variable categórica
    selected_cat_var = st.sidebar.selectbox('Seleccione la variable categórica para el análisis', options=categorical_cols)

    if selected_cat_var:
        # Calcular la tabla de frecuencias para la variable seleccionada
        freq_table = data_cat[selected_cat_var].value_counts().reset_index()
        freq_table.columns = [selected_cat_var, 'Frecuencia']

        # Slider para que el usuario elija la frecuencia mínima
        min_freq = st.sidebar.slider('Frecuencia mínima de las categorías a visualizar',
                                     min_value=int(freq_table['Frecuencia'].min()),
                                     max_value=int(freq_table['Frecuencia'].max()),
                                     value=int(freq_table['Frecuencia'].min()))

        # Filtrar las categorías según la frecuencia mínima seleccionada
        filtered_table = freq_table[freq_table['Frecuencia'] >= min_freq]

        # Seleccionar todas las categorías filtradas de una sola vez
        if st.sidebar.button('Seleccionar todas las categorías'):
            selected_categories = list(filtered_table[selected_cat_var])  # Seleccionar todas las categorías
        else:
            # Mostrar las categorías filtradas en un multiselect para que el usuario elija las que quiere graficar
            selected_categories = st.sidebar.multiselect(f'Selecciona las categorías de {selected_cat_var} para graficar',
                                                         options=filtered_table[selected_cat_var],
                                                         default=[])

        # Asegurarse de que se tengan categorías seleccionadas antes de graficar
        if selected_categories:
            # Filtrar la tabla por las categorías seleccionadas
            final_table = filtered_table[filtered_table[selected_cat_var].isin(selected_categories)]

            # Mostrar la tabla final filtrada en la aplicación
            st.write(f"Categorías seleccionadas de '{selected_cat_var}' con al menos {min_freq} ocurrencias:")
            st.write(final_table)

            # Selección del tipo de gráfico
            chart_type = st.sidebar.radio("Elige el tipo de gráfico",
                                          ('Barras', 'Pastel', 'Ambos'))

            # Gráfico de Barras (Barplot)
            if chart_type == 'Barras' or chart_type == 'Ambos':
                st.subheader('Gráfico de Barras')
                fig, ax = plt.subplots(figsize=(15, 8)) 
                ax.bar(final_table[selected_cat_var], final_table['Frecuencia'], width=0.6)
                # Rotar etiquetas del eje x
                ax.set_xticklabels(final_table[selected_cat_var], rotation=90, ha='right')
                ax.set_xlabel(f'Nombre del {selected_cat_var}')
                ax.set_ylabel('Frecuencia')
                ax.set_title(f'Distribución de Frecuencias de {selected_cat_var}')
                st.pyplot(fig)

            # Gráfico de Pastel
            if chart_type == 'Pastel' or chart_type == 'Ambos':
                st.subheader('Gráfico de Pastel')
                fig, ax = plt.subplots()
                # Pie Plot
                ax.pie(final_table['Frecuencia'], labels=final_table[selected_cat_var], autopct='%1.1f%%', startangle=90)
                ax.axis('equal')  # Función que tiene que ver con la forma
                st.pyplot(fig)




# Regresión Logística 
elif analysis_type == "Regresión Logística":
    st.header('Análisis de Regresión Logística')

    # Selección de las variables independientes y dependiente
    independent_vars = st.sidebar.multiselect('Selecciona las variables independientes:', options=numeric_cols)
    dependent_var = st.sidebar.selectbox('Selecciona la variable dependiente:', options=numeric_cols)

    if independent_vars and dependent_var:
        X = numeric_data[independent_vars]

        # Calcular la mediana de la variable dependiente
        median_value = np.median(numeric_data[dependent_var])

        # Binarizar la variable dependiente usando la mediana
        y = numeric_data[dependent_var].apply(lambda x: 1 if x > median_value else 0)

        # Escalar las variables independientes
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Dividir en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y
        )

        # Verificar si hay valores NaN o Inf 
        if np.isnan(X_train).any() or np.isinf(X_train).any():
            st.error("Error: X_train contiene valores NaN o Inf.")
            st.stop()

        if np.isnan(y_train).any() or np.isinf(y_train).any():
            st.error("Error: y_train contiene valores NaN o Inf.")
            st.stop()

        # Verificar las clases en y_train
        unique_classes = np.unique(y_train)
        if len(unique_classes) < 2:
            st.error("Error: y_train debe contener al menos dos clases diferentes.")
            st.stop()

        # Crear y entrenar el modelo de regresión logística
        log_model = LogisticRegression()
        log_model.fit(X_train, y_train)

        # Predicciones
        y_pred = log_model.predict(X_test)

        # Calcular métricas
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        sensibilidad = recall_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, output_dict=True)

        # Mostrar los resultados
        st.write('### Resultados del Análisis de Regresión Logística:')
        st.write(f'Exactitud del modelo: {accuracy:.2f}')
        st.write(f'Precision del modelo: {precision:.2f}')
        st.write(f'Sensibilidad del modelo: {sensibilidad:.2f}')
        st.write('Matriz de Confusión:')
        st.write(conf_matrix)

        st.write('Reporte de Clasificación:')
        st.dataframe(pd.DataFrame(class_report).transpose())

        # Visualización: Matriz de Confusión con Seaborn
        fig, ax = plt.subplots()
        sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d', cbar=False, ax=ax)
        ax.set_xlabel('Predicción')
        ax.set_ylabel('Valor Real')
        ax.set_title('Matriz de Confusión')
        st.pyplot(fig)

Writing app.py
